## Setup

The ONC Python package can be installed via pip: `!pip install onc`

I store my ONC token in a `.netrc` file in my user directory so I don't risk sharing it.

```
machine data.oceannetworks.ca
login <user_here_or_leave_blank>
password <token_here>
```

In [1]:
from datetime import datetime, timezone, timedelta
from onc import ONC
from netrc import netrc

_,__, token = netrc().authenticators('data.oceannetworks.ca')
onc = ONC(token = token)

## Helper Functions

In [2]:
def find_nav_dev_code(pvcs_dev_code: str) -> str:
    """
    Identify a NAV device code based on a ferry's PVCS device code.
    The unique device code for a PVCS should remain the same by vessel (not route).
    :param pvcs_dev_code: An ONC Oceans 3.0 device code (e.g. PVCS01).

    :return: The unique device code for the NAV.
    """
    date_to = datetime.now(timezone.utc)
    date_from = date_to - timedelta(seconds = 5)
    locs = onc.getLocationsTree({'deviceCode': pvcs_dev_code,
                                  'dateFrom': date_from.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z',
                                  'dateTo': date_to.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'})
    loc_code = locs[0]['children'][0]['children'][0]['children'][0]['locationCode']
    nav_iters = [loc_code] + ['.'.join((loc_code,N)) for N in ['N1','N2','N3']]
    nav_dev_code = None
    for nav_iter in nav_iters:
        try:
            nav_info = onc.getDevices({'locationCode': nav_iter,
                                             'deviceCategoryCode': 'NAV',
                                              'dateFrom': date_from.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z',
                                              'dateTo': date_to.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'})
            nav_dev_code = nav_info[0]['deviceCode']
            return nav_dev_code
        except:
            continue
    if nav_dev_code is None:
        raise RuntimeError(f'No NAV devices found for location codes: {nav_iters}')

In [3]:
def get_realtime(device_code: str, jump_back: int = 3):
    """
    Get the last data point from an ONC device. If no data exists, then an HTTPError will propagate from the ONC Python package.

    :param device_code: An ONC device code tied to a specific instrument and serial number.
    :param jump_back: The number of seconds to look back in case there is a delay.
    :return: A dictionary of dictionaries containing the most recent data point.
    """
    date_to = datetime.now(timezone.utc)
    date_from = date_to - timedelta(seconds = jump_back)

    json_data = onc.getScalardata({'deviceCode': device_code,
                                   'dateFrom': date_from.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z',
                                   'dateTo': date_to.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z',
                                   'qualityControl': 'raw'})

    inst_data = json_data['sensorData']

    data_dict = {}
    for var_data in inst_data:

        # Build variable name from more descriptive sensor name.
        var_name = var_data['sensorName']
        var_name = var_name.replace(' ', '_').lower()
        var_name = var_name.replace('(', '')
        var_name = var_name.replace(')', '')

        data = var_data['data']
        times = data['sampleTimes']
        vals = data['values']

        # Only return the most recent value.
        recent_time = times[-1]
        recent_val = vals[-1]

        data_dict[var_name] = {'time': recent_time, 'value':recent_val}

    return data_dict

## Sampling Code

system_state == 0 means system is not sampling.

valve_position == 0 means the valve is not open to environment.

valve_position == 2 means the valve is set to allow cycling of freshwater from freshwater tank.

outlet_flow <= 1 means the system is struggling to produce flow, potentially indicating a blockage.

pump_current <= 1 means the pump may be failing.


In [4]:
PVCS_DEV_CODE = 'PVCS01'  # The device code for the Pump and Valve Controller System on the SOVI as of 2025-12-23. "Should" be connected to the SOVI.

#NAV_DEV_CODE = 'HEMISPHEREGNSSGPS-C14360018202663'  # The device code for the GNSS unit on the SOVI as of 2025-12-23.
NAV_DEV_CODE = find_nav_dev_code(PVCS_DEV_CODE) # Seeks out the GNSS device code based on the PVCS device code. Makes multiple try/except requests so it adds about 2 seconds to script time.

In [5]:

# Acquire Data
nav = get_realtime(NAV_DEV_CODE)
pvcs = get_realtime(PVCS_DEV_CODE)

# Obtain key info.
system_state = int(pvcs['system_state']['value'])
valve_position = int(pvcs['valve_position']['value'])
outlet_flow = pvcs['outlet_flow']['value']
pump_current = pvcs['pump_current']['value']

# Conditions for a bad or closed flowthrough system state.
no_run_states = [0,2]
bad_flow = 1.0
bad_pump = 1.0

if (system_state in no_run_states or
        valve_position in no_run_states or
        outlet_flow <= bad_flow or
        pump_current <= bad_pump):

    # Don't run IFCB.
    # Code for handling not running the IFCB goes here.

    print(f"System State: {system_state}")
    print(f"Valve Position: {valve_position}")
    print(f"Outlet Flow: {outlet_flow}")
    print(f"Pump Current: {pump_current}")
    print('System state NOT OK for IFCB operation.')

elif (system_state not in no_run_states and
      valve_position not in no_run_states and
      outlet_flow > bad_flow and
      pump_current > bad_pump):

    # Run IFCB
    # Code for running the IFCB goes here.

    print(f"System State: {system_state}")
    print(f"Valve Position: {valve_position}")
    print(f"Outlet Flow: {outlet_flow}")
    print(f"Pump Current: {pump_current}")
    print('System state OK for IFCB operation.')

else:
    # Error handling in case system state doesn't return a 0 or 1.
    raise RuntimeError('No handling for unknown conditionals.')


System State: 1
Valve Position: 1
Outlet Flow: 5.483
Pump Current: 2.1
System state OK for IFCB operation.
